# Model prompting with installed Ollama models

In [159]:
%%capture output
%run ./06_0_prompt_creation.ipynb

In [160]:
import pandas as pd

embedding = "te3l" # / "te3s"
grobid_model = "full_model"
no_prev_chunking = True

annotated = True
corrected_statements = True

path = f"../data/dfs{'/annotated_data' if annotated else ''}{'/corrected_statements' if corrected_statements else ''}/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_chunk_info.pkl"
print(path)

# read the dataframe from a pickle file
df = pd.read_pickle(path)

../data/dfs/annotated_data/corrected_statements/te3l_no_prev_chunking/full_model/ReferenceErrorDetection_data_with_chunk_info.pkl


In [161]:
df[df['Corrected Statement'].notna()]

,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Statement with Citation,Corrected Statement,Citation Statement Section,...,Reference Article PDF Available,Reference Article Retracted,Reference Article Downloaded,Label,Explanation,Error Type,Supporting Sentences,Suited for Task,Top_3_Chunk_IDs,Top_3_Chunk_Texts
84,"Smith & Cumberledge, 2020",c059,10.1126/science.aaf6846,ATP as a biological hydrotrope,No,Yes,Biology,We reconstituted FUS compartments in vitro at ...,We reconstituted FUS compartments in vitro at ...,Methods,...,Yes,No,Yes,Fully substantiate,Method,NaN,NaN,Not enough understanding,"[9e220442-a0ee-4a2b-9dde-624dc4e5c653, 6f8c4d3...","[In tissues containing mitochondria, where the..."
86,"Smith & Cumberledge, 2020",c084,10.1038/ncomms13828,Vacancy-induced dislocations within grains for...,No,Yes,Chemistry,other parameters including Gruneisen parameter...,"…, other parameters including ... and Poisson ...",Results,...,Yes,No,Yes,Unsubstantiate,The cited article gives an equation while the ...,3.2 - Misunderstanding-partsup,NaN,Yes,"[0e02390c-1c54-45c4-8ef1-58452b54f30a, dfd738d...",[À 2a3ð1 þ 3aÞ ð1 þ 23aÞ 2 Â 1 À 2d þ 2d 2 ln ...
88,Correction,c046,10.3390/jcdd8050052,Peptidic Connexin43 Therapeutics in Cardiac Re...,No,Yes,Medicine,Other investigations of Peptide5 in preclinica...,Other investigations of Peptide5 in preclinica...,Related Work,...,Yes,No,Yes,Unsubstantiate,Wrong reference,4 - Unwanted,NaN,Yes,"[65ce124a-90ee-434b-9bc5-8b50140a5c89, afddb72...",[Human renal proximal tubule epithelial cells ...
89,"Lee & Lee, 1999",c048,10.1111/j.1346-8138.1993.tb01394.x,Association of Generalized granuloma annulare ...,No,Yes,Biology,T-cell subpopulations in granuloma annulare we...,"…, and the number of S100 positive dendritic c...",Discussion,...,Yes,No,Yes,Unsubstantiate,Over-generalization to another tissue,3.2 - Misunderstanding-partsup,NaN,Yes,"[b5b31fa7-d7ea-4c77-994d-81669c2fafb6, 2c943cd...","[Using this method, S100 positive, basal melan..."
90,"Lee & Lee, 1999",c049,10.1111/j.1346-8138.1993.tb01389.x,Autoantibodies in scleroderma,No,Yes,Biology,"In early studies, anti-Scl-70 was identified i...","In early studies, anti-Scl-70 was ..., but, wi...",Related Work,...,Yes,No,Yes,Unsubstantiate,Wrong number (75% vs. 77%).,2 - Copy-Paste-Error,NaN,Yes,"[d5437287-bb0c-434e-a1f1-a70253098740, 88a8688...",[Nucleolar staining without any associated pat...
97,Correction,c046,10.3390/jcdd8050052,Peptidic Connexin43 Therapeutics in Cardiac Re...,No,Yes,Medicine,Other investigations of Peptide5 in preclinica...,Other investigations of Peptide5 in preclinica...,Related Work,...,Yes,No,Yes,Unsubstantiate,Wrong reference,4 - Unwanted,NaN,Yes,"[9e8b50ef-37ee-4b24-afac-c11734e4e6ed, ef362ed...",[Peptides and peptide-derived molecules target...
103,"Smith & Cumberledge, 2020",c062,10.1126/science.aag3009,Mapping the human DC lineage through the integ...,No,Yes,Biology,Pitt-Hopkins syndrome (PHS) is characterized b...,"..., which encodes the E2-2 transcription fact...",Background,...,Yes,No,Yes,Fully substantiate,NaN,NaN,"[""However, E2–2 likely represents only one of ...",Not enough understanding,"[c4b94938-22d6-4a4a-9cd5-465f2144a4b4, 927680e...","[Some of the players are known, including pDCe..."
106,"Smith & Cumberledge, 2020",c065,10.1126/sciadv.1600983,Snowball Earth climate dynamics and Cryogenian...,No,Yes,Environment,"In the MDV, 1000 km to the north and 1.8 km lo...","In the MDV (Antarctic McMurdo Dry Valleys), 10...",Background,...,Yes,No,Yes,Unsubstantiate,No mention of cyanobacteria or photosynthesis,3.2 - Misunderstanding-partsup,NaN,Yes,"[9c987fd5-7a90-43c5-8d14-dbb8086c664b, 92be724...",[Ecology of moderately halophilic bacteria. Th...
112,"Smith & Cumberledge, 2020",c071,10.1126/sciadv.1501645,Self-powered integrated microfluidic point-of-...,No,Yes,Biology,We used new nucleic acid amplification technol...,We used new nucleic acid amplif

In [163]:
import ollama

def send_prompt(prompt, model):
    response = ollama.chat(model=model, messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

In [164]:
import time

def prompting_model(df, model, skip_existing=False, save_intermediate_results=False, save_every=10):
    print(f"Prompting model: {model}")

    # Create a new column in the dataframe to store the responses
    if 'Model Classification' not in df.columns:
        df['Model Classification'] = None

    # Iterate through the dataframe
    for index, row in df.iterrows():
        if row['Reference Article Downloaded'] == 'Yes':
            if skip_existing and pd.notna(row['Model Classification']):
                print(f"Skipping existing classification for index {index}")
                continue
            start_time = time.time()
            print(f"Processing: " + row['Reference Article ID'])

            # Create the prompt
            prompt = create_prompt(row, corrected_statements=corrected_statements)
            
            # Send the prompt and get the response
            response = send_prompt(prompt, model)
            
            # Save the response to the new column
            df.at[index, 'Model Classification'] = response

            if save_intermediate_results and index % save_every == 0:
                df.to_pickle(f"../data/dfs{'/annotated_data' if annotated else ''}{'/corrected_statements' if corrected_statements else ''}/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_prompt_results_{model.replace(':','.')}_intermed.pkl")
            end_time = time.time()
            print(f"Took {round(end_time - start_time, 2)} seconds")
            print("==================================")
    return df

In [165]:
%run ../scripts/load_df_for_analysis.py

In [166]:
# This function reads the df with the prompt results for the non-annotated data 
# adds the annotation information to the df
# and removes all model classification values for the data rows which were annotated
# so that only these values will be overwritten in the next prompting
def prepare_df2_reprompt_annotated_data(df_annotated):
    df2 = sort_df(pd.read_pickle(f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_prompt_results_{model.replace(':','.')}.pkl"))

    # Remove rows from df_annotated where "Suited for Task" is "Added"
    df_annotated = sort_df(df_annotated[df_annotated['Suited for Task'] != "Added"])

    # Ensure df2 and df_annotated have the same number of rows
    assert len(df2) == len(df_annotated), "Mismatch in the number of rows between df2 and df_annotated"

    # Add missing columns from df_annotated to df2 with empty values
    for column in df_annotated.columns:
        if column not in df2.columns:
            df2[column] = None

    # Add missing columns from df_annotated to df2
    for index, row in df2.iterrows():
        assert row['Reference Article ID'] == df_annotated.iloc[index]['Reference Article ID'], f"Mismatch in Reference Article ID at index {index}"
        assert row['Citing Article ID'] == df_annotated.iloc[index]['Citing Article ID'], f"Mismatch in Citing Article ID at index {index}"

        # Fill the rows with values from df_annotated
        for column in df_annotated.columns:
            if column in df2.columns:
                df2.at[index, column] = df_annotated.iloc[index][column]

    # Reorder the columns in df2 to match the order of columns in df_annotated
    # Keep the columns that df2 has additionally on the very right
    common_columns = df_annotated.columns
    additional_columns = [col for col in df2.columns if col not in common_columns]
    df2 = df2[list(common_columns) + additional_columns]

    # Remove all model classification values for the data rows which were annotated
    for index, row in df2.iterrows():
        if row['Reference Article Downloaded'] == 'Yes' and not pd.isna(row['Citation Function: Main']):
            # Set the model classification to None for the annotated rows
            df2.at[index, 'Model Classification'] = None

    return df2

In [167]:
df2 = prepare_df2_reprompt_annotated_data(df)
display(df2.head())
display(df2[df2['Corrected Statement'].notna()].head())

,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Statement with Citation,Corrected Statement,Citation Statement Section,...,Reference Article Retracted,Reference Article Downloaded,Label,Explanation,Error Type,Supporting Sentences,Suited for Task,Top_3_Chunk_IDs,Top_3_Chunk_Texts,Model Classification
0,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Others have aimed to reduce irreversibility or...,NaN,NaN,...,No,Yes,Unsubstantiate,Irrelevant,NaN,NaN,NaN,"[e3f23238-791d-4749-81e5-fd3c55c2686b, af18d41...",[Nomenclature\nES:Expert system ANN:Artificial...,Here is the answer in JSON format:\n\n```\n{\n...
1,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Some researchers have also studied various hea...,NaN,NaN,...,No,Yes,Unsubstantiate,Irrelevant,NaN,NaN,NaN,"[66e4b712-fc01-4965-a835-5bd9dcdb40cb, 5957595...",[Heat transfer improvement of water/single-wal...,"Here is the JSON response:\n\n```\n{\n ""label..."
2,PubPeer,c002,10.1155/2022/4601350,Oxidative Potential and Nanoantioxidant Activi...,Yes,Yes,Chemistry,The relative content of total flavonoids in th...,NaN,NaN,...,No,Yes,Unsubstantiate,Irrelevant,NaN,NaN,NaN,"[e53a16b7-7a90-4656-b2c6-ad1c105cc881, 01a8b83...",[The correspondence curve for our photochemica...,"Here is my assessment:\n\n```\n{\n""label"": ""Un..."
3,PubPeer,c003,10.1155/2022/2408685,The Choice of Anesthetic Drugs in Outpatient H...,Yes,Yes,Medicine,Research has shown that remimazolam tosylate e...,NaN,NaN,...,No,Yes,Unsubstantiate,Irrelevant,NaN,NaN,NaN,"[934c19f1-dd5d-4427-bf55-8a4d1aeb9f5a, ec73144...",[Determination of the median effective concent...,Here is my response in JSON format:\n\n```\n{\...
4,PubPeer,c004,10.1155/2022/4783847,A Fault-Tolerant Structure for Nano-Power Comm...,Yes,Yes,Engineering,if the efficiency of the routing algorithm is ...,NaN,NaN,...,No,Yes,Unsubstantiate,Irrelevant,NaN,NaN,NaN,"[f38e8c2d-1f59-4744-9cdd-7dd775bca1ed, c2fe9ce...","[In the table, China's intermediary centrality...","Here is my evaluation:\n\n```\n{\n ""label"": ""..."


,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Statement with Citation,Corrected Statement,Citation Statement Section,...,Reference Article Retracted,Reference Article Downloaded,Label,Explanation,Error Type,Supporting Sentences,Suited for Task,Top_3_Chunk_IDs,Top_3_Chunk_Texts,Model Classification
82,Correction,c046,10.3390/jcdd8050052,Peptidic Connexin43 Therapeutics in Cardiac Re...,No,Yes,Medicine,Other investigations of Peptide5 in preclinica...,Other investigations of Peptide5 in preclinica...,Related Work,...,No,Yes,Unsubstantiate,Wrong reference,4 - Unwanted,NaN,Yes,"[65ce124a-90ee-434b-9bc5-8b50140a5c89, afddb72...",[Human renal proximal tubule epithelial cells ...,None
83,Correction,c046,10.3390/jcdd8050052,Peptidic Connexin43 Therapeutics in Cardiac Re...,No,Yes,Medicine,Other investigations of Peptide5 in preclinica...,Other investigations of Peptide5 in preclinica...,Related Work,...,No,Yes,Unsubstantiate,Wrong reference,4 - Unwanted,NaN,Yes,"[9e8b50ef-37ee-4b24-afac-c11734e4e6ed, ef362ed...",[Peptides and peptide-derived molecules target...,None
87,"Lee & Lee, 1999",c048,10.1111/j.1346-8138.1993.tb01394.x,Association of Generalized granuloma annulare ...,No,Yes,Biology,T-cell subpopulations in granuloma annulare we...,"…, and the number of S100 positive dendritic c...",Discussion,...,No,Yes,Unsubstantiate,Over-generalization to another tissue,3.2 - Misunderstanding-partsup,NaN,Yes,"[b5b31fa7-d7ea-4c77-994d-81669c2fafb6, 2c943cd...","[Using this method, S100 positive, basal melan...",None
88,"Lee & Lee, 1999",c049,10.1111/j.1346-8138.1993.tb01389.x,Autoantibodies in scleroderma,No,Yes,Biology,"In early studies, anti-Scl-70 was identified i...","In early studies, anti-Scl-70 was ..., but, wi...",Related Work,...,No,Yes,Unsubstantiate,Wrong number (75% vs. 77%).,2 - Copy-Paste-Error,NaN,Yes,"[d5437287-bb0c-434e-a1f1-a70253098740, 88a8688...",[Nucleolar staining without any associated pat...,None
101,"Smith & Cumberledge, 2020",c059,10.1126/science.aaf6846,ATP as a biological hydrotrope,No,Yes,Biology,We reconstituted FUS compartments in vitro at ...,We reconstituted FUS compartments in vitro at ...,Methods,...,No,Yes,Fully substantiate,Method,NaN,NaN,Not enough understanding,"[9e220442-a0ee-4a2b-9dde-624dc4e5c653, 6f8c4d3...","[In tissues containing mitochondria, where the...",None


In [169]:
models = ["llama3.1:70b", "llama3.1:405b", "llama3.3"]
model = models[0]

In [170]:
%%time

df2 = prompting_model(df2, model, skip_existing=True, save_intermediate_results=True, save_every=5)

Prompting model: llama3.1:70b
Skipping existing classification for index 0
Skipping existing classification for index 1
Skipping existing classification for index 2
Skipping existing classification for index 3
Skipping existing classification for index 4
Skipping existing classification for index 5
Skipping existing classification for index 6
Skipping existing classification for index 7
Skipping existing classification for index 8
Skipping existing classification for index 9
Skipping existing classification for index 10
Skipping existing classification for index 11
Skipping existing classification for index 12
Skipping existing classification for index 13
Skipping existing classification for index 14
Skipping existing classification for index 15
Skipping existing classification for index 16
Skipping existing classification for index 18
Skipping existing classification for index 19
Skipping existing classification for index 20
Skipping existing classification for index 21
Skipping exist

In [171]:
df2.to_pickle(f"../data/dfs{'/annotated_data' if annotated else ''}{'/corrected_statements' if corrected_statements else ''}/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_prompt_results_{model.replace(':','.')}.pkl")